<a href="https://colab.research.google.com/github/romanzm13/Analisis_Datos_Defunciones_Jalisco_por_Region/blob/main/FiltradoDatosDefunciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install xlrd==1.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: xlrd
    Found existing installation: xlrd 2.0.1
    Uninstalling xlrd-2.0.1:
      Successfully uninstalled xlrd-2.0.1


In [2]:
from numpy import count_nonzero,arange,array,dot,asarray,zeros,apply_along_axis,around,sort,shape,savetxt,array_equal,max,argmin,argmax,fill_diagonal,ones,argsort,std,diag,random
from matplotlib.pyplot import plot,figure,title,legend,xlabel,ylabel,grid,axhline,axvline,savefig,imshow,show,scatter,hist,bar,subplot,subplots,cm,Normalize,text
from math import sqrt
import pandas as pd
from datetime import datetime,timedelta
from scipy.stats.mstats import mquantiles
from statistics import mean,median,mode
from scipy.stats import poisson
from copy import deepcopy
pd.options.display.max_rows=10

Importar base de datos

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Los datos utilizados fueron tomados del conjunto de datos abiertos del INEGI. En concreto, la fuente es la siguiente:

Subsistema de Información Demográfica y Social. (2021). *Defunciones registradas (mortalidad general)*. Recuperado de https://www.inegi.org.mx/programas/mortalidad/#Datos_abiertos

Limpieza de la base de datos

In [4]:

df=pd.read_csv('/content/drive/MyDrive/DATOS_INEGI_DEFUNCIONES/conjunto_de_datos_defunciones_registradas_2021.csv')

print("Tamaño de la base de datos:")
print(df.shape)
print("Información general de los datos:")
print(df.info())
#Cuántos diferentes valores posee cada variable
print("Cantidad de valores diferentes para cada variable:")
print(df.nunique())
#Verificar si hay valores nulos
print("Valores nulos en cada variable")
print(df.isnull().sum())

Tamaño de la base de datos:
(1122249, 59)
Información general de los datos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1122249 entries, 0 to 1122248
Data columns (total 59 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   ent_regis   1122249 non-null  int64 
 1   mun_regis   1122249 non-null  int64 
 2   ent_resid   1122249 non-null  int64 
 3   mun_resid   1122249 non-null  int64 
 4   tloc_resid  1122249 non-null  int64 
 5   loc_resid   1122249 non-null  int64 
 6   ent_ocurr   1122249 non-null  int64 
 7   mun_ocurr   1122249 non-null  int64 
 8   tloc_ocurr  1122249 non-null  int64 
 9   loc_ocurr   1122249 non-null  int64 
 10  causa_def   1122249 non-null  object
 11  lista_mex   1122249 non-null  object
 12  sexo        1122249 non-null  int64 
 13  edad        1122249 non-null  int64 
 14  dia_ocurr   1122249 non-null  int64 
 15  mes_ocurr   1122249 non-null  int64 
 16  anio_ocur   1122249 non-null  int64 
 17  dia_regi

In [5]:
#Construcción del nuevo data frame
#Sólo nos quedaremos con los campos de entidad de residencia, municipio de residencia, año de ocurrencia, lugar de ocurrencia, asistencia médica
#sitio de ocurrencia, afiliación de servicios de salud, edad agrupada
#Eliminar campos
df1=df[['ent_resid','mun_resid','anio_ocur','lugar_ocur','asist_medi','sitio_ocur','derechohab','edad_agru']]
#causa de defunción (lista_mexicana): 'lista_mex'
#Se desea filtrar aquellas observaciones que correspondan al año de ocurrencia de 2021
filtro1=df1['anio_ocur']==2021
df2=df1[filtro1]
#Sólo usaremos los datos de Jalisco
filtro2=df2['ent_resid']==14
df3=df2[filtro2]
#Eliminar los registros en los que el municipio es '0' ó '999', que corresponden a Municipio no especificado
filtro3=df3['mun_resid']!=0
df4=df3[filtro3]
filtro4=df4['mun_resid']!=999
df5=df4[filtro4]
df5.drop(['ent_resid', 'anio_ocur'],axis=1,inplace=True)
#Borrar registros con valor nulo
df5=df5.dropna()
print("Tamaño de la base de datos:")
print(df5.shape)
print("Información general de los datos:")
print(df5.info())
print("Cantidad de valores diferentes para cada variable:")
print(df5.nunique())

Tamaño de la base de datos:
(74233, 6)
Información general de los datos:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 74233 entries, 57 to 1122232
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   mun_resid   74233 non-null  int64
 1   lugar_ocur  74233 non-null  int64
 2   asist_medi  74233 non-null  int64
 3   sitio_ocur  74233 non-null  int64
 4   derechohab  74233 non-null  int64
 5   edad_agru   74233 non-null  int64
dtypes: int64(6)
memory usage: 4.0 MB
None
Cantidad de valores diferentes para cada variable:
mun_resid     125
lugar_ocur     11
asist_medi      3
sitio_ocur     13
derechohab     10
edad_agru      28
dtype: int64


<ipython-input-5-689cab5b7ab4>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5.drop(['ent_resid', 'anio_ocur'],axis=1,inplace=True)


In [6]:
#Importar los datos sobre la regionalización de Jalisco
df_reg=pd.read_excel("/content/drive/MyDrive/DATOS_INEGI_DEFUNCIONES/Regionalizacion_Jalisco.xlsx",sheet_name="por_municipio")

#df_reg['Municipio']=df_reg['Municipio'].astype(str)
#df_reg['Region']=df_reg['Region'].astype(str)

print("Información general de los datos:")
print(df_reg.info())
#Cuántos diferentes valores posee cada variable
print("Cantidad de valores diferentes para cada variable:")
print(df_reg.nunique())
#Sólo nos interesa quedarnos con la 'Clave' y el 'No_Region'
df_reg.drop(['Municipio','Habitantes','Region'],axis=1,inplace=True)

Información general de los datos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125 entries, 0 to 124
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Clave       125 non-null    int64 
 1   Municipio   125 non-null    object
 2   Habitantes  125 non-null    int64 
 3   No_Region   125 non-null    int64 
 4   Region      125 non-null    object
dtypes: int64(3), object(2)
memory usage: 5.0+ KB
None
Cantidad de valores diferentes para cada variable:
Clave         125
Municipio     125
Habitantes    125
No_Region      12
Region         12
dtype: int64


In [7]:
#Unir los datos procedentes de los dos dataframes
df_ext=pd.merge(df5, df_reg, left_on='mun_resid', right_on='Clave')
#Eliminar la columna Clave de este nuevo dataframe
df_ext.drop(['Clave'],axis=1,inplace=True)
print("Información general de los datos:")
print(df_ext.info())
#Cuántos diferentes valores posee cada variable
print("Cantidad de valores diferentes para cada variable:")
print(df_ext.nunique())
#Exportar datos filtrados como archivo csv
df_ext.to_csv("Datos_filtrados_defunciones_2021.csv",index=False)

Información general de los datos:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 74233 entries, 0 to 74232
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   mun_resid   74233 non-null  int64
 1   lugar_ocur  74233 non-null  int64
 2   asist_medi  74233 non-null  int64
 3   sitio_ocur  74233 non-null  int64
 4   derechohab  74233 non-null  int64
 5   edad_agru   74233 non-null  int64
 6   No_Region   74233 non-null  int64
dtypes: int64(7)
memory usage: 4.5 MB
None
Cantidad de valores diferentes para cada variable:
mun_resid     125
lugar_ocur     11
asist_medi      3
sitio_ocur     13
derechohab     10
edad_agru      28
No_Region      12
dtype: int64
